# Notebook for running Runge-Kutta4 solver on rate equations

Made as a summer student at CERN, at the [CRIS](http://isolde-cris.web.cern.ch/isolde-cris/index.html) experiment at [ISOLDE](http://isolde.web.cern.ch/) for simulation of spontaneous emission, absorption and stimulated emission in an atom. This can be related in rate equations for how the radiation interacts with matter.
1. Spontaneous emission 
2. Absorption
3. Stimulated emssion


This can be related trough the general rate equations with energy levels $i$ and $k$ where $i < k$

$\frac{dN_2}{dt} = N_1B_{12}\rho(\omega_{21}) - N_2B_{21} \rho(\omega_{21})- N_2A_{21}$


$\frac{dN_1}{dt} = -N_1B_{12}\rho(\omega_{21}) + N_2B_{21} \rho(\omega_{21}) +N_2A_{21} = - \frac{dN_2}{dt} $

This can be simulated for multiple levels, as long as the Einstein coefficients are available, it they are not, I have made the assumption that no transition is possible for consistency with experimental data available from [NIST Atomic Spectra Database Lines Data](https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra=Sn+i&limits_type=0&low_w=&upp_w=&unit=1&de=0&java_out=Make+Grotrian+Diagram&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on)

In [2]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [3]:
# import stuff
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import re
import seaborn as sns
from scipy.constants import pi, speed_of_light, hbar, Boltzmann, Planck, elementary_charge
from scipy import signal
import readstuff as rs


In [4]:
params = {'legend.fontsize': 'large',
          'figure.figsize': (15, 10),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)
plt.rcParams['figure.dpi']= 300

# Making a solver including pumping

Some information we need:

$laser_{linewidth} \approx 10$ GHz

$D_{beam} = 2-3$ mm

$f_{pumping} = 100 $ Hz (how many pulses the laser fires)

$t_{pump} \approx 10$ ns (length of a single laser pulse)


## Calculate the pumping coefficient $B$

Using that $\frac{A_{21}}{B_{21}} = \frac{\hbar\omega_{21}^3}{\pi^2c^3}$

and 

$g_1B_{12} = g_2B_{21}$

In [5]:
def getB(A,freq):
    return (A*pi**2*speed_of_light**3)/(hbar*(2*pi*freq)**3)

## Calculate $\rho_{max}$

In [6]:
def get_rho_tot(pulse_energy,beam_D,beam_duration):
    return pulse_energy/((pi*(0.5*beam_D))**2*beam_duration*speed_of_light)

def get_rho_max(laser_linewidth,pulse_energy,beam_D,beam_duration):
    return (1/(pi*laser_linewidth))*np.sqrt(np.log(2)/pi)*get_rho_tot(pulse_energy,beam_D,beam_duration)

### Find the time of flight (when the voltage is 40 kV)

In [7]:
mass = 119.9021947*1.6605e-27 # [kg]
E = 6.4087065e-15 #40000*1.602176565e-19 # [J]
l = 1.20 # [m]
v = np.sqrt(2*E/mass) # [m/s]
tof = l/v # [s]
# tof
stps = int(tof/1e-9)
TOF = stps*1e-9
# TOF
TOF2 = 1.35e-5
stps2 = int(TOF2/1e-9)
# stps2

## Find all the laser parameters needed
 $\lambda = v/f \implies f = v/\lambda \implies f = c/\lambda$

In [9]:
laser_width = 10*1e9 # [Hz]
pumpf = 100
tot_pump_per_sec = 1 - pumpf*10*1e-9 
beaml = 10*1e-9 # [s]
pumpav = tot_pump_per_sec/pumpf # how long th elaser is shut off at a time
pump_t0 = tof/2 # when the laser first fires
pE = 1e-3 # [J] pulse energy, measure instead
beamD = 3*1e-3 # [m] 

### Read in initial and final populations

In [14]:
def rd_fin(file,col=['level','pop']):
    pd.read_csv(col,names=col2,delimiter=';',float_precision='high').apply(pd.to_numeric,errors='coerce').sort_values(by='level').reset_index(drop=True)

In [15]:
def rd_init(file,col =['level','xs']):
    df = rd_fin(file,col)
    return rs.renorm(df).sort_values(by='level').reset_index(drop=True)

## Find the relative strength of decay

In [10]:
# Normalize the Einsteincoefficients
def normal_Einstein(dfa):
    df = dfa.copy()
    nlist = []
    for u in range(np.max(df.k)+1):
        upper = np.asarray(df[df.k == u].Aki)        
        if len(upper) > 0:
            nupper = upper/np.sum(upper)
            nlist.append(nupper)
    df['nAki'] = np.concatenate(np.array(nlist)).ravel()
    return df

## Find maximum possible distribution of each level as $t\rightarrow \infty$

In [11]:
def find_max(initialpopulation, dfa_norm):
    maxpop = np.array(initialpopulation['pop'].copy(),dtype=np.float128)
    dfa = dfa_norm.copy().sort_values(by='Ek').reset_index(drop=True)
    # first, find the levels that can decay multiple times trough different levels cascade
    decaymore = np.full(len(dfa_norm),False)
    
    for k, (ak, akrow) in enumerate(dfa.iterrows()):
        for i, (ai, airow) in enumerate(dfa.iterrows()):
            if k!=i and int(akrow['i']) == int(airow['k']):
                decaymore[ak] = True
                print('k',ak,'   ',int(akrow['k']),'>',int(akrow['i']),'>', int(airow['k']),'>',int(airow['i']))

    dfa['decaysmore'] = decaymore

    #let the upper level decay first
    fra = np.array(dfa['k']) # levels to decay from
    til = np.array(dfa['i']) # levels to decay to
    nAki = np.array(dfa['nAki'], dtype = np.float128)

    lendiff = fra.size # length of dN/dt 
    mfra = np.max(fra) + 1 # highest upper level + 1 for indexing
    mtil = np.max(til) + 1 # highest lower level + 1 for indexing
    armax = np.arange(mfra)
    armin = np.arange(mtil)
    gridfra = (np.mgrid[:mfra,:lendiff] == fra)[0] # grids for uniquely grouping transmission
    gridtil = (np.mgrid[:mtil,:lendiff] == til)[0]

    def f(N):  # N = population
        dN = np.zeros(len(N),dtype=np.float128)
        diff = N[fra]*nAki # dN/dt change in population
        # summing dN/dt for each unique level
        temp1 = (gridfra*diff).sum(axis=1) 
        temp2 = (gridtil*diff).sum(axis=1)
        dN[armax] -= temp1 # decay from upper levels
        dN[armin] += temp2 # decay to lower levels
        return dN
    
    for k, (a,arow) in enumerate(dfa.iterrows()):
        # letting the levels transmit to lower levels that can decay
        if arow.decaysmore == True:
            diff = initialpopulation['pop'][int(arow.k)]*arow.nAki
            maxpop[int(arow.k)] -= diff
            maxpop[int(arow.i)] += diff
            print(int(arow.k),int(arow.i))
    endpop = maxpop + f(maxpop)
    plt.vlines(initialpopulation['level'], np.zeros(len(endpop)), endpop, colors='k', linestyles='solid')
    plt.ylabel('Distribution')
    plt.xlabel(r'Wavenumber $[cm^{-1}]$')
    return endpop, dfa


## Find pumping routes

In [12]:
def pumpxtoy(x,y,df,tol=0.05):
    pxy = df[(((df.i == y)& (df.nAki > tol)) | ((df.i == x) & (df.nAki < 0.5))) & (df.nAki != 1)].sort_values(by='k').reset_index(drop=True)
    return pxy[pxy.duplicated(subset='k', keep=False)].sort_values(by=['Ek','i','nAki'])

## Calculate wavelengths

In [13]:
def add_wave(df):
    df['wavelength'] = normal_emc3['wavelength'] = (1/(df.Ek-df.Ei))*1e-2*1e9
    return df

## Solve the rate equations

In [ ]:
def solver_vec(df_A,initialpop,stop,steps,pumpab=[1,2,3,5],pumpstart=0.5*tof, pumpduration=beaml,pumpoff=pumpav,laser_linewidth=laser_width,pulse_energy=pE,beam_D=beamD,pumping=False):
    time = np.linspace(0,stop,steps)
    dt = time[1] - time[0]; dt2 = dt*0.001
    laserfreq = np.asarray(speed_of_light/(df_A.loc[pumpab].wavelength*1e-9),dtype=np.float128)
    print(laserfreq)
#     if abs(np.log10(stop) - np.log10(dt)) < 3:
#         print(dt)
#         raise ValueError("More steps needed for enough accuracy!")
    rho_on = get_rho_max(laser_linewidth,pulse_energy,beam_D,pumpduration)
#     rho_on = get_new_rho(nu=laserfreq)
    population = np.array(initialpop['pop'].copy(),dtype=np.float128)
    p_ev = np.zeros((steps,len(population))) # get the time evolution of the population
    print('rho',rho_on)
    fra = np.array(df_A['k']) # levels to decay from
    til = np.array(df_A['i']) # levels to decay to
    Aki = np.array(df_A['Aki'], dtype = np.float128)
    Akino = Aki.copy()
    Akino[1:6] = 0
    print(Akino)
    print(Aki)
    lendiff = fra.size # length of dN/dt 
    mfra = np.max(fra) + 1 # highest upper level + 1 for indexing
    mtil = np.max(til) + 1 # highest lower level + 1 for indexing
    armax = np.arange(mfra); armin = np.arange(mtil)
    gridfra = (np.mgrid[:mfra,:lendiff] == fra)[0] # grids for uniquely grouping transmission
    gridtil = (np.mgrid[:mtil,:lendiff] == til)[0]

    # setting levels for pumping and finding the B coefficients
    A_ba = df_A.iloc[pumpab]
    print('aki', A_ba['Aki'])
    B_ba = np.asarray(getB(np.asarray(A_ba['Aki']),laserfreq),dtype=np.float128)

    # degeneracy of from lower level a, to upper level b
    gb = 2*newn2.sort_values(by='Ek').reset_index(drop=True).loc[pumpab].u_J + 1 
    ga = 2*newn2.sort_values(by='Ek').reset_index(drop=True).loc[pumpab].l_J + 1#g = 2J+1
    B_ab = np.asarray((gb/ga)*B_ba,dtype=np.float128)# replace with g factor when time comes
    a = np.asarray(A_ba['i'],dtype=int); b = np.asarray(A_ba['k'],dtype=int)
    print(a,b)
    l_fra = np.array(A_ba['i']) # levels to pump from
    l_til = np.array(A_ba['k']) # levels to pump to
    l_Aki = np.array(A_ba['Aki'], dtype = np.float128)
    print('fra', l_fra, 'til ',l_til)
    l_lendiff = l_fra.size # length of dN/dt 
    l_mfra = np.max(l_fra) + 1 # highest upper level + 1 for indexing
    l_mtil = np.max(l_til) + 1 # highest lower level + 1 for indexing
    l_armax = np.arange(l_mfra); l_armin = np.arange(l_mtil)
    l_gridfra = (np.mgrid[:l_mfra,:l_lendiff] == l_fra)[0] # grids for uniquely grouping transmission
    l_gridtil = (np.mgrid[:l_mtil,:l_lendiff] == l_til)[0]
    print('l_mfra ',l_mfra,'lmtil',l_mtil)
    print('l_armax',l_armax,'l_armin',l_armin)
    print('l_gridfra',l_gridfra)
    print('l_gridtil',l_gridtil)
    indexl = np.asarray(np.unique(np.append(a,b)),dtype=int)
    print(indexl)
    
    
    totsteps = int((stop-pumpduration)/dt + pumpduration/dt2)
    pop_ev = np.zeros((len(indexl),totsteps+1))
    print(pop_ev.shape)
    pop_ev[:,0] = population[indexl]
    print(pop_ev.shape)
    def f(N,laserlock=False):
        if laserlock == True:
            diff = N[fra]*Akino # pumping
        else:
            diff = N[fra]*Aki # dN/dt change in population
        dN = np.zeros(len(N),dtype=np.float128)
        # summing dN/dt for each unique level
        temp1 = (gridfra*diff).sum(axis=1) # subtracting 
        temp2 = (gridtil*diff).sum(axis=1) # adding     
        dN[armax] -= temp1 # decay from upper levels
        dN[armin] += temp2 # decay to lower levels
        return dN # = dN/dt
    
    def fl(N): 
        Na = N[a]
        Nb = N[b] 
        pump = Na*B_ab*rho_on #- Nb*B_ba*rho_on
        dN = f(N,True)
        dNcopy = np.zeros(len(dN))
        pumpa = (l_gridfra*pump).sum(axis=1)
        pumpb = (l_gridtil*pump).sum(axis=1)
        dNcopy[l_armax] -= pumpa
        dNcopy[l_armin] += pumpb
        dN += dNcopy
        return dN
    
    def rk4(N,laser=False):
        if laser == True:
            k1 = dt2*fl(N)
            k2 = dt2*fl(N + 0.5*k1)
            k3 = dt2*fl(N + 0.5*k2)
            k4 = dt2*fl(N + k3)
        else:
            k1 = dt*f(N)
            k2 = dt*f(N + 0.5*k1)
            k3 = dt*f(N + 0.5*k2)
            k4 = dt*f(N + k3)
        u_new = N + (1/6.0)*(k1 + 2*k2 + 2*k3 + k4)
        return u_new
    
    t = int(0)
    tid = 0
    t_vec = np.zeros(totsteps+1)
    
    if pumping == True:  # implement pumping
        while tid < pumpstart:
            
            population = rk4(population)
            t += 1
            pop_ev[:,t] = population[indexl]
            t_vec[t] = tid
            tid += dt
            
        while pumpstart <= tid <= pumpstart + pumpduration:
            population = rk4(population,True)
            t += 1
            pop_ev[:,t] = population[indexl]
            t_vec[t] = tid
            tid += dt2
        if abs(np.sum(population) - np.sum(initialpop['pop'])) > 1e-7:
            print(abs(np.sum(population) - np.sum(initialpop['pop'])))
            raise ValueError("Distribution not conserved")
        print(abs(np.sum(population) - np.sum(initialpop['pop'])))
        t_vec = np.trim_zeros(t_vec,'b')
        return population, pop_ev[:,:len(t_vec)], t_vec
        while tid < stop:
            population = rk4(population)
            t += 1
            pop_ev[:,t] = population[indexl]
            t_vec[t+1] = tid
            tid += dt 
        if pumping == False:
            if abs(np.sum(population) - np.sum(initialpop['pop'])) > 1e-7:
                print(abs(np.sum(population) - np.sum(initialpop['pop'])))
                raise ValueError("Distribution not conserved")
            return population

    
    print(abs(np.sum(population) - np.sum(initialpop['pop'])))
    if pumping == True:
        return population, pop_ev, t_vec
    else:
        return population